**Creating a csv file which has the path of images and its name**

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import pandas as pd

# Load MoveNet Thunder Model
model_url = "https://tfhub.dev/google/movenet/singlepose/thunder/4"
movenet = hub.load(model_url)

# Function to extract keypoints from an image
def extract_keypoints(image_path):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0).astype(np.int32)

    outputs = movenet.signatures["serving_default"](input=img)
    keypoints = outputs['output_0'].numpy().reshape(-1, 3)[:, :2]  # Extract (x, y)

    return keypoints.flatten()

# Directory where images are stored (organized by pose names)
images_dir = "images"

data, labels = [], []
for pose in os.listdir(images_dir):
    pose_path = os.path.join(images_dir, pose)
    if os.path.isdir(pose_path):
        for img_file in os.listdir(pose_path):
            img_path = os.path.join(pose_path, img_file)
            keypoints = extract_keypoints(img_path)
            data.append(keypoints)
            labels.append(pose)  # Use folder name as label

# Save dataset to CSV
df = pd.DataFrame(data)
df["label"] = labels
df.to_csv("yoga_keypoints.csv", index=False)
print("✅ Dataset Created: yoga_keypoints.csv")


✅ Dataset Created: yoga_keypoints.csv


In [2]:
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Load dataset
df = pd.read_csv("yoga_keypoints.csv")
X, y = df.iloc[:, :-1].values, df["label"].values

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train Random Forest model
model = Pipeline([
    ("scaler", StandardScaler()),
    ("classifier", RandomForestClassifier(n_estimators=200, random_state=42))
])
model.fit(X_train, y_train)

# Save trained model
joblib.dump(model, "yoga_pose_model.pkl")
print(f"✅ Model trained with {model.score(X_test, y_test) * 100:.2f}% accuracy.")
print("✅ Model saved as yoga_pose_model.pkl")


✅ Model trained with 100.00% accuracy.
✅ Model saved as yoga_pose_model.pkl


In [6]:
import cv2
import numpy as np
import joblib
import tensorflow_hub as hub
import pandas as pd
from scipy.spatial import distance

# Load MoveNet Thunder Model
model_url = "https://tfhub.dev/google/movenet/singlepose/thunder/4"
movenet = hub.load(model_url)

# Load trained model
classifier = joblib.load("yoga_pose_model.pkl")

# Load dataset to get ideal keypoints for each pose
df = pd.read_csv("yoga_keypoints.csv")
ideal_keypoints = df.groupby("label").mean().to_dict(orient="index")  # Average keypoints for each pose

# Function to extract keypoints from a frame
def extract_keypoints_from_frame(frame):
    img = cv2.resize(frame, (256, 256))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=0).astype(np.int32)

    outputs = movenet.signatures["serving_default"](input=img)
    keypoints = outputs["output_0"].numpy().reshape(-1, 3)[:, :2]

    return keypoints.flatten()

# Function to calculate corrections & rating
def calculate_corrections(detected_keypoints, ideal_keypoints):
    corrections = []
    keypoint_labels = [
        "Nose", "Left Eye", "Right Eye", "Left Ear", "Right Ear", "Left Shoulder", "Right Shoulder",
        "Left Elbow", "Right Elbow", "Left Wrist", "Right Wrist", "Left Hip", "Right Hip",
        "Left Knee", "Right Knee", "Left Ankle", "Right Ankle"
    ]

    # Check if predicted pose exists in ideal_keypoints
    if predicted_pose not in ideal_keypoints:
        print(f"⚠️ Warning: No ideal keypoints found for {predicted_pose}")
        return ["Error: Ideal keypoints not found"], 0  # Error message and rating 0

    ideal_pose_keypoints = np.array(ideal_keypoints[predicted_pose])  # Convert to NumPy array
    total_diff = 0

    for i, label in enumerate(keypoint_labels):
        detected = detected_keypoints[i * 2:i * 2 + 2]  # (x, y)
        ideal = ideal_pose_keypoints[i * 2:i * 2 + 2]  # (x, y)

        if len(detected) == 2 and len(ideal) == 2:
            diff = distance.euclidean(detected, ideal)
            total_diff += diff
            if diff > 20:  # Threshold for correction
                corrections.append(f"Adjust {label} closer to ideal position")

    avg_diff = total_diff / len(keypoint_labels)
    rating = max(10 - int(avg_diff / 5), 1)  # Convert avg_diff to rating (1-10 scale)

    return corrections, rating

# Capture image from webcam, process it, and display output in VS Code
cap = cv2.VideoCapture(0)
ret, frame = cap.read()
cap.release()

if ret:
    keypoints = extract_keypoints_from_frame(frame).reshape(1, -1)
    predicted_pose = classifier.predict(keypoints)[0]

    corrections, rating = calculate_corrections(keypoints.flatten(), ideal_keypoints[predicted_pose])

    print(f"✅ Predicted Pose: {predicted_pose}")
    print(f"✅ Rating: {rating}/10")
    print("✅ Corrections Needed:")
    for correction in corrections:
        print(f"   - {correction}")

else:
    print("❌ Could not capture an image from the webcam.")


⚠️ Warning: No ideal keypoints found for baddha konasana
✅ Predicted Pose: baddha konasana
✅ Rating: 0/10
✅ Corrections Needed:
   - Error: Ideal keypoints not found
